In [1]:
import numpy as np
import lmfit
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio
import glob
from tqdm import tqdm
import os
from scipy.constants import physical_constants
import subprocess
from scipy.optimize import curve_fit

## Defining functions

In [2]:
def openArchives(caminho):
    """
    Opens and processes a list of files from a specified path, sorting them by modification time. 

    Each file is expected to be in a tab-separated format, with filenames containing a numeric value 
    used for ordering and indexing. Extracts these values from filenames and reads data into a list.

    Parameters:
    caminho (str): The path pattern of files to be processed (e.g., './data/*.txt').

    Returns:
    tuple: 
        - dados (np.ndarray): A multidimensional array containing the data from each file.
        - values (list of float): A list of numeric values extracted from each filename.
    """

    # Get the list of files and sort them by modification time
    arquivos = glob.glob(caminho)
    
    # Sort files by modification time, from oldest to newest
    arquivos_sorted = sorted(arquivos, key=lambda x: os.stat(x).st_mtime)
    
    values = []  # List to store the values extracted from filenames
    dados = []   # List to store the data loaded from each file
    
    # Loop through each sorted file
    for each in tqdm(arquivos_sorted, desc='Processing items'):
        try:
            # Extract numerical value from filename, assumed to be the second-to-last element
            value = float(each.split('_')[each.count('_') - 1])
            # Load the data from the file, assuming tab-separated values
            data = np.loadtxt(each, delimiter='\t')
            
            # Append extracted value and data to the respective lists
            values.append(value)
            dados.append(data)
        except Exception as e:
            # Print error message if an issue arises with loading a file
            print(f'Há um erro de importação em: {each}, Erro: {e}')

    # Convert list of data arrays into a single multidimensional numpy array
    dados = np.array(dados)
    
    # Return the processed data array and list of extracted values
    return dados, values

In [3]:
def rmvBaseline(x, y):
    """
    Removes the baseline from a set of y-values by calculating a linear baseline based on 
    the slope and intercept derived from the mean values of the first and last points.

    Parameters:
    x (np.ndarray): The x-axis values, typically representing an independent variable (e.g., time or wavelength).
    y (np.ndarray): The y-axis values, from which the baseline will be removed.

    Returns:
    np.ndarray: The y-values with the baseline subtracted.
    """
    
    n_of_points = 5  # Number of points to use from the start and end for baseline calculation

    # Calculate the slope of the baseline using the average of the first and last few points
    slope = (y[:n_of_points].mean() - y[-n_of_points:].mean()) / (x[:n_of_points].mean() - x[-n_of_points:].mean())
    
    # Calculate the intercept of the baseline
    intercept = y[-n_of_points:].mean() - (slope * x[-n_of_points:].mean())
    
    # Calculate the baseline as a linear function based on the slope and intercept
    baseline = (slope * x) + intercept
    
    # Return the y-values with the calculated baseline subtracted
    return y - baseline

In [ ]:
def plotOnePL(data):
    """
    Plots photoluminescence (PL) data in an interactive graph.

    Parameters:
    -----------
    data : numpy.ndarray
        2D array with wavelengths (nm) in the first column and counts in the second.

    Functionality:
    --------------
    - Converts wavelength to energy (eV).
    - Plots energy (X-axis) vs. counts (Y-axis) as a scatter plot.
    """

    # Convert wavelength (nm) to energy (eV)
    energy = 1239.84 / data[:, 0]
    
    # Get counts (intensity) from the data
    counts = data[:, 1]
    
    # Create the plot figure
    fig = go.Figure()
    
    # Add scatter plot of data points to the figure
    fig.add_trace(go.Scatter(x=energy, y=counts, mode='markers', name='PL Data'))
    
    # Set the title of the plot
    fig.update_layout(title="PL Data")
    
    # Display the plot
    fig.show()

In [ ]:
# Lorentzian function definition
def lorentzian(x, center, amplitude, sigma):
    """
    Defines a Lorentzian function, commonly used to model spectral peaks.

    Parameters:
    -----------
    x : np.ndarray
        The independent variable, such as energy or wavelength.
    center : float
        The central position of the peak.
    amplitude : float
        The peak height or maximum intensity.
    sigma : float
        The half-width at half-maximum (HWHM) of the peak.

    Returns:
    --------
    np.ndarray
        The Lorentzian function values at each point in x.
    """
    return (amplitude / np.pi) * (sigma / ((x - center) ** 2 + sigma ** 2))


# Gaussian function definition
def gaussian(x, center, area, fwhm):
    """
    Defines a Gaussian function, frequently used to model symmetrical peaks in spectral data.

    Parameters:
    -----------
    x : np.ndarray
        The independent variable, such as energy or wavelength.
    center : float
        The central position of the peak.
    area : float
        The total area under the Gaussian curve.
    fwhm : float
        The full width at half maximum (FWHM) of the peak.

    Returns:
    --------
    np.ndarray
        The Gaussian function values at each point in x.
    """
    return (area * np.exp((-4 * np.log(2) * (x - center) ** 2) / fwhm ** 2)) / (fwhm * np.sqrt(np.pi / (4 * np.log(2))))

In [ ]:
def area_of_curve(height, width):
    """
    Estimates the area of a curve for an initial guess based on its height and width.

    Parameters:
    -----------
    height : float
        The maximum height of the curve.
    width : float
        The width of the curve, often taken as the full width at half maximum (FWHM) for peak shapes.

    Returns:
    --------
    float
        The estimated area of the curve.
    """

    # Calculate the area using the given height and width, with a scaling factor for approximation
    area = height * width * np.sqrt((np.pi / 2))
    
    return area


In [ ]:
def combined_model(x, *params):
    y = np.zeros_like(x)
    for i in range(2): #The value in range define the number of curves that you wanna use to fit
        center = params[i*3]
        area = params[i*3+1]
        fwhm = params[i*3+2]
        y += gaussian(x, center, area, fwhm)
    #offset = params[-1]
    return y #+ offset

In [4]:
def zeeman(x, g):
    """
    Calculates the Zeeman effect energy shift based on the applied magnetic field.

    The Zeeman effect describes the splitting of a spectral line into multiple components
    in the presence of a static magnetic field. This function computes the energy shift 
    for a given magnetic field strength.

    Parameters:
    -----------
    x : np.ndarray or float
        The magnetic field strength (in Tesla) for which to calculate the energy shift.
    g : float
        The g-factor, which characterizes the magnetic moment of the particle.

    Returns:
    --------
    np.ndarray or float
        The energy shift (in eV) corresponding to the applied magnetic field.
    """
    return g * physical_constants['Bohr magneton in eV/T'][0] * x


In [ ]:
def is_increasing(lst):
    """
    Checks if a list of numbers is strictly increasing.

    A list is considered strictly increasing if each element is less than the next one.

    Parameters:
    -----------
    lst : list
        A list of numeric values to be checked.

    Returns:
    --------
    bool
        True if the list is strictly increasing, False otherwise.
    """
    return all(x < y for x, y in zip(lst, lst[1:]))


## Using the functions

In [ ]:
# Call the openArchives function to load data and values
dados, values = openArchives(caminho)

# Print the extracted values and shape of the loaded data
print("Extracted Values:", values)
print("Shape of Loaded Data:", dados.shape)

In [ ]:
# Sample values and data (replace with your actual data)
values = [1.0, 2.0, 3.0, 2.0]  # Example of a non-increasing list
dados = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])  # Sample 2D array

# Check if values are increasing
if not is_increasing(values):
    # Flip the values and corresponding data if not increasing
    values = np.flip(values)
    dados = np.flip(dados, axis=0)

# Print the results after potentially flipping
print("Values:", values)
print("Dados:", dados)

In [ ]:
# Ensure that initial_guess is within the bounds
initial_guess = [1.400, 5, 0.003,   # Peak 0 (center, amplitude, sigma)
                 1.410, 15, 0.004]    # Peak 1
                 # Offset

# Bounds: Ensure bounds are correctly set and consistent with initial_guess
bounds = ([1.380, 0, 0,   # Lower bounds for Peak 0
           1.385, 0, 0,   # Lower bounds for Peak 1
          ], 
          [1.420, np.inf, np.inf,  # Upper bounds for Peak 0
           1.440, np.inf, np.inf,   # Upper bounds for Peak 1
          ])              # Upper bound for Offset

# Initialize lists to store results
# params_ajustados = pd.DataFrame()

num_graph = 0
fitted_spectras_1 = go.Figure()
frames = []
resultados = []
params_example_data = pd.DataFrame()

# Iterate over each spectrum in the data
for espectro in tqdm(example_data, desc='Fitting spectra.'):
    try:
        # Convert wavelength to energy and extract counts
        energy = 1239.84 / espectro[:, 0]
        counts = espectro[:, 1]
        # Remove defective data points
        filter_peaks = (energy >= 1.35) & (energy <= 1.45)
        energy = energy[filter_peaks]
        counts = counts[filter_peaks]
        # Remove baseline (you can keep this function unchanged)
        # counts = rmvBaseline(energy, counts)
        # Fit the spectrum using curve_fit
        popt, pcov = curve_fit(combined_model, energy, counts, p0=initial_guess, bounds=bounds)
        # Store the best-fit parameters
        resultados.append(popt)
        # Update initial guess with new parameters for the next iteration
        initial_guess = popt
        # Prepare data for plotting individual Gaussian components
        components = [gaussian(energy, *popt[i*3:i*3+3]) for i in range(2)]
        # offset = popt[-1]

        # Update DataFrame with new parameters
        new_row = {f'peak{i}_{param}': popt[i*3+j] for i in range(2) for j, param in enumerate(['center', 'area', 'fwhm'])}
        # new_row['offset'] = offset

        if params_example_data.empty:
            params_example_data = pd.DataFrame([new_row])
        else:
            params_example_data = pd.concat([params_example_data, pd.DataFrame([new_row])], ignore_index=True)
        param_step = value_example_data[num_graph]
        # Increment num_graph for the next iteration
        
        # Prepare plot data
        frames_data = [
            {'x': energy, 'y': counts, 'mode': 'markers', 'name': 'Raw Data'},
            {'x': energy, 'y': combined_model(energy, *popt), 'mode': 'lines', 'name': f'Fit at {param_step} T'},
            {'x': energy, 'y': components[0], 'mode': 'lines', 'name': 'Gaussian 1'},
            {'x': energy, 'y': components[1], 'mode': 'lines', 'name': 'Gaussian 2'}
        ]
        frame = go.Frame(data=frames_data, name=f'Frame {num_graph}')
        frames.append(frame)
        num_graph += 1
    except ValueError as e:
        print(f'Error: {e}')

params_example_data['Magnetic Field'] = value_example_data
####################  Plot the fittings ##############################

# Add the initial curves to the plot. In this case, all are the same (raw data)
fitted_spectras_1.add_traces([
    go.Scatter(x=energy, y=counts, mode='markers', name='Raw data', marker=dict(color='blue')),  # Raw data as markers
    go.Scatter(x=energy, y=counts, mode='lines', name='Raw data', line=dict(color='#DB0700')),  # Raw data as first line
    go.Scatter(x=energy, y=counts, mode='lines', name='Raw data', line=dict(color='#059D32')),  # Raw data as second line
    go.Scatter(x=energy, y=counts, mode='lines', name='Raw data', line=dict(color='#F27F1B'))   # Raw data as third line
])

# Define the frames (animation frames) using the content of 'frames' created earlier
fitted_spectras_1.frames = frames

# Configure the layout of the figure, including sliders to navigate between frames
fitted_spectras_1.update_layout(
    sliders=[{
        'steps': [{
            'args': [[frame.name], {'frame': {'redraw': True}, 'mode': 'immediate'}],  # Configures the slider behavior when selecting a frame
            'label': f'Index {i}',  # Label displayed on the slider
            'method': 'animate'  # Defines that the method to be called is 'animate'
        } for i, frame in enumerate(frames)],  # Creates an entry for each existing frame
        'x': 0.55,  # Horizontal position of the slider
        'len': 1.3,  # Length of the slider
        'currentvalue': {'font': {'size': 12}, 'prefix': None, 'visible': True, 'xanchor': 'center'},  # Configures the current value displayed on the slider
        'pad': {'b': 10, 't': 50},  # Spacing around the slider
        'xanchor': 'center',  # Horizontal alignment of the slider
        'yanchor': 'top'  # Vertical alignment of the slider
    }],
    # Add play/pause buttons
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 300, 'redraw': True}, 'fromcurrent': True, 'mode': 'immediate'}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'fromcurrent': True}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 1.25,
        'xanchor': 'right',
        'y': 0.5,
        'yanchor': 'top'
    }],
    # title_text="Data and Fits",  # Title of the graph
    showlegend=True,  # Show the graph legend
    margin=dict(r=100, l=50, t=10, b=2),
    plot_bgcolor='white',
    xaxis={'title': {'text': 'Energy (eV)', 'font': {'size': 18, 'family': 'Arial', 'color': 'black'}}, 'linecolor': 'black', 'linewidth': 2.5, 'ticks': 'outside', 'tickwidth': 2, 'range': [1.35, 1.45], 'mirror': True, 'showgrid': False},
    yaxis={'title': {'text': 'Intensity', 'font': {'size': 18, 'family': 'Arial', 'color': 'black'}}, 'linecolor': 'black', 'linewidth': 2.5, 'ticks': 'outside', 'tickwidth': 2, 'mirror': True, 'showgrid': False},
    
    width=980,  # Set figure width
    height=900,  # Set figure height
    autosize=False,  # Disable autosizing
)  

# Display the final figure, with the configuration defined above
fitted_spectras_1.show()


## Plot the results

This is specific for energy, but you can change to plot area or fwhm

In [ ]:
# Create a figure for plotting
excampo = go.Figure()

# Add traces for each Lorentzian sigma parameter
excampo.add_trace(go.Scatter(
    x=params_example_data['Magnetic field'], 
    y=params_example_data['peak0_sigma'], 
    mode='markers', 
    name='Lorentzian 1', 
    marker=dict(color='#059D32')  # Green color for the first Lorentzian
))

excampo.add_trace(go.Scatter(
    x=params_example_data['Magnetic field'], 
    y=params_example_data['peak1_sigma'], 
    mode='markers', 
    name='Lorentzian 2', 
    marker=dict(color='#F27F1B')  # Orange color for the second Lorentzian
))

# Update the layout of the figure
excampo.update_layout(
    title='Energy vs Magnetic Field',
    xaxis_title='Magnetic Field (T)',
    yaxis_title='Energy (eV)',
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black', 
        linewidth=2, 
        ticks='outside', 
        tickwidth=2, 
        mirror=True, 
        showgrid=False
    ),
    yaxis=dict(
        linecolor='black', 
        linewidth=2, 
        ticks='outside', 
        tickwidth=2, 
        mirror=True, 
        showgrid=False
    ),
)

# Show the figure
excampo.show()


In [ ]:
# Extract peak energies for negative and positive magnetic fields
x0_energy_neg = list(params_ajustados['peak1_center'][params_ajustados['Magnetic field'] < 0])
x0_energy_pos = list(params_ajustados['peak1_center'][params_ajustados['Magnetic field'] > 0])

# Reverse the negative energy list
x0_energy_neg.reverse()

# Calculate the difference between the two energy lists
difference = list(map(lambda x, y: x - y, x0_energy_neg, x0_energy_pos))

# Prepare the Zeeman model for fitting
zeeman_shift = lmfit.Model(zeeman)

# Perform the fit, ensure to set initial guess for 'g' if necessary
result_zeeman = zeeman_shift.fit(data=difference, x=params_ajustados['Magnetic field'][params_ajustados['Magnetic field'] > 0], g=-4)

# Print the results
print("R-squared:", result_zeeman.rsquared)
print("Fitted parameters:")
print(result_zeeman.params)

# Plot the fit results
result_zeeman.plot_fit()


## Export fits

In [ ]:
# Create a directory to store the frames
if not os.path.exists("frames"):
    os.makedirs("frames")

# Loop through each frame and save it as an image
for i, frame in enumerate(fitted_spectras.frames):
    # Update the figure to display the current frame
    fitted_spectras.update(frames=[frame])
    
    # Save the current frame as an image
    filename = f"frames/frame_{i:03d}.png"  # Format the filename with leading zeros
    pio.write_image(fitted_spectras, filename, format='png')  # Save the figure, not just the frame
